# San Francisco City Employee Salary

# 1 | Introduction

<img src='https://cdn.pixabay.com/photo/2017/03/18/14/56/panorama-2154194_960_720.jpg'>

## What topic does the dataset cover?

SF Salaries Dataset contains San Francisco City employee salary data. 

This data contains the names, job title, and compensation for San Francisco City employees on an annual basis from 2011 to 2014.

## Objectives of the notebook:

Explore and preprocessing the data set for clustring models.

# 2 | Importing libraries

In [1]:
# Importing Necessary Libraries.

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

# 3 | Reading the dataset

In [3]:
sf_data = pd.read_csv('data/Salaries.csv')

In [4]:
sf_data.shape

(148654, 13)

In [5]:
sf_data.sample(5)

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
42558,42559,Eric Montalvo,Firefighter,110280.11,1679.99,11578.71,37660.68,123538.81,161199.49,2012,NaN,San Francisco,NaN
147220,147221,Racquel M Mcgee,Communications Dispatcher 1,2051.47,52.90,99.70,22.04,2204.07,2226.11,2014,NaN,San Francisco,PT
50904,50905,Bing Tam,Personal Property Auditor,79417.0,0.0,480.0,34791.29,79897.00,114688.29,2012,NaN,San Francisco,NaN
122516,122517,Allen J Lee,Chemist,94984.02,0.0,0.0,34621.44,94984.02,129605.46,2014,NaN,San Francisco,FT
68331,68332,Valerie Neil,Communications Dispatcher 2,11550.74,0.0,322.44,941.07,11873.18,12814.25,2012,NaN,San Francisco,NaN


# 4 | Summary Of Numerical Features

In [6]:
sf_data.describe().T.style.set_properties(**{'background-color': 'gray','color': 'white','border-color': 'white'})

,count,mean,std,min,25%,50%,75%,max
Id,148654.000000,74327.500000,42912.857795,1.000000,37164.250000,74327.500000,111490.750000,148654.000000
TotalPay,148654.000000,74768.321972,50517.005274,-618.130000,36168.995000,71426.610000,105839.135000,567595.430000
TotalPayBenefits,148654.000000,93692.554811,62793.533483,-618.130000,44065.650000,92404.090000,132876.450000,567595.430000
Year,148654.000000,2012.522643,1.117538,2011.000000,2012.000000,2013.000000,2014.000000,2014.000000
Notes,0.000000,nan,nan,nan,nan,nan,nan,nan


# 5 | Data Cleaning

### 5.1 create a helper function for get information of columns

In [7]:
def summary_statistic(df) -> pd.DataFrame:

        """
        function input: it get a pandas dataframe as a csv file
        output: return: retuen a information about columns as pandas dataframe
        """

        feature_describe = df.describe().T.reset_index().rename(
                               columns={'index':'feature'}).drop(columns='count')

        feature_info = pd.concat([df.dtypes,
                                   df.nunique(),
                                   df.isna().sum(),
                                   df.count()], axis=1,
                    keys=['type', 'count_unique', 'count_nan', 'count']).reset_index().rename(columns={'index':'feature'})

        summary_statistic_result = feature_info.merge(feature_describe, how='left', on='feature')

        return summary_statistic_result

In [8]:
summary_statistic(sf_data)

,feature,type,count_unique,count_nan,count,mean,std,min,25%,50%,75%,max
0,Id,int64,148654,0,148654,74327.500000,42912.857795,1.00,37164.250,74327.50,111490.750,148654.00
1,EmployeeName,object,110811,0,148654,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,JobTitle,object,2159,0,148654,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BasePay,object,109900,605,148049,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,OvertimePay,object,66555,0,148654,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,OtherPay,object,84968,0,148654,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Benefits,object,99635,36159,112495,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,TotalPay,float64,138486,0,148654,74768.321972,50517.005274,-618.13,36168.995,71426.61,105839.135,567595.43
8,TotalPayBenefits,float64,142098,0,148654,93692.554811,62793.533483,-618.13,44065.650,92404.09,132876.450,567595.43
9,Year,int64,4,0,148654,2012.522643,1.117538,2011.00,2012.000,2013.00,2014.000,2014.00


### 5.2 Converting Data types of all columns.

In [9]:
sf_data['Id'] = sf_data['Id'].astype('int32')
sf_data['EmployeeName'] = sf_data['EmployeeName'].astype('string')
sf_data['JobTitle'] = sf_data['JobTitle'].astype('string')
sf_data['TotalPay'] = sf_data['TotalPay'].astype('float32')
sf_data['TotalPayBenefits'] = sf_data['TotalPayBenefits'].astype('float32')
sf_data['Agency'] = sf_data['Agency'].astype('string')
sf_data['Status'] = sf_data['Status'].astype('string')

sf_data['BasePay'] = pd.to_numeric(sf_data['BasePay'], errors = 'coerce')
sf_data['OvertimePay'] = pd.to_numeric(sf_data['OvertimePay'], errors = 'coerce')
sf_data['OtherPay'] = pd.to_numeric(sf_data['OtherPay'], errors = 'coerce')
sf_data['Benefits'] = pd.to_numeric(sf_data['Benefits'], errors = 'coerce')


sf_data.dtypes

Id                    int32
EmployeeName         string
JobTitle             string
BasePay             float64
OvertimePay         float64
OtherPay            float64
Benefits            float64
TotalPay            float32
TotalPayBenefits    float32
Year                  int64
Notes               float64
Agency               string
Status               string
dtype: object

### 5.3 Removing columns "Notes & Status".

In [10]:
sf_data = sf_data.drop(columns = ['Notes', 'Status'])

sf_data.columns

Index(['Id', 'EmployeeName', 'JobTitle', 'BasePay', 'OvertimePay', 'OtherPay',
       'Benefits', 'TotalPay', 'TotalPayBenefits', 'Year', 'Agency'],
      dtype='object')

### 5.4 Number of Null values in each columns.

In [11]:
# sf_data.isna().sum()
summary_statistic(sf_data)[['feature','count_nan']]

,feature,count_nan
0,Id,0
1,EmployeeName,0
2,JobTitle,0
3,BasePay,609
4,OvertimePay,4
5,OtherPay,4
6,Benefits,36163
7,TotalPay,0
8,TotalPayBenefits,0
9,Year,0


### 5.5 Checking any duplicate rows.

In [12]:
sf_data[sf_data.duplicated()]

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Agency


### 5.6 Number of Unique values in each columns.

In [13]:
sf_data.nunique()
summary_statistic(sf_data)[['feature','count_unique']]

,feature,count_unique
0,Id,148654
1,EmployeeName,110811
2,JobTitle,2159
3,BasePay,109489
4,OvertimePay,65998
5,OtherPay,83225
6,Benefits,98465
7,TotalPay,138318
8,TotalPayBenefits,141809
9,Year,4


In [14]:
summary_statistic(sf_data)

,feature,type,count_unique,count_nan,count,mean,std,min,25%,50%,75%,max
0,Id,int32,148654,0,148654,74327.500000,42912.857795,1.000000,37164.250000,74327.500000,111490.750000,148654.0000
1,EmployeeName,string,110811,0,148654,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,JobTitle,string,2159,0,148654,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BasePay,float64,109489,609,148045,66325.448841,42764.635495,-166.010000,33588.200000,65007.450000,94691.050000,319275.0100
4,OvertimePay,float64,65998,4,148650,5066.059886,11454.380559,-0.010000,0.000000,0.000000,4658.175000,245131.8800
5,OtherPay,float64,83225,4,148650,3648.767297,8056.601866,-7058.590000,0.000000,811.270000,4236.065000,400184.2500
6,Benefits,float64,98465,36163,112491,25007.893151,15402.215858,-33.890000,11535.395000,28628.620000,35566.855000,96570.6600
7,TotalPay,float32,138318,0,148654,74768.101562,50515.890625,-618.130005,36168.994141,71426.609375,105839.134766,567595.4375
8,TotalPayBenefits,float32,141809,0,148654,93693.054688,62792.984375,-618.130005,44065.650391,92404.089844,132876.449219,567595.4375
9,Year,int64,4,0,148654,2012.522643,1.117538,2011.000000,2012.000000,2013.000000,2014.000000,2014.0000


### 5.7 Now we can take the case of "JobTitle" column.

In [15]:
# Removing leading and Trailing white spaces in "JobTitle" column.
sf_data['JobTitle'] = sf_data['JobTitle'].str.strip()

# Lets see the Top 60 Jobtitles.
sf_data['JobTitle'].value_counts().head(60)

Transit Operator                        7036
Special Nurse                           4389
Registered Nurse                        3736
Public Svc Aide-Public Works            2518
Police Officer 3                        2421
Custodian                               2418
TRANSIT OPERATOR                        2388
Firefighter                             2359
Recreation Leader                       1971
Patient Care Assistant                  1945
Deputy Sheriff                          1933
Police Officer                          1476
SPECIAL NURSE                           1402
Public Service Trainee                  1328
REGISTERED NURSE                        1219
Police Officer 2                        1141
Attorney (Civil/Criminal)               1126
Porter                                  1095
Sergeant 3                              1047
General Laborer                         1033
EMT/Paramedic/Firefighter                918
Gardener                                 894
Parking Co

In [16]:
sf_data.loc[sf_data['JobTitle'].str.lower().str.contains('transit operator'), 'JobTitle'].value_counts()

# Here I decided to combine these 2.

Transit Operator    7036
TRANSIT OPERATOR    2388
Name: JobTitle, dtype: Int64

In [17]:
sf_data.loc[sf_data['JobTitle'].str.lower().str.contains('nurse'), 'JobTitle'].value_counts()

# Here I decided to group these different "Nurse Jobtitles" into a single Jobtitle as "Nurse".

Special Nurse                4389
Registered Nurse             3736
SPECIAL NURSE                1402
REGISTERED NURSE             1219
Licensed Vocational Nurse     707
Nurse Practitioner            674
Nurse Manager                 262
NURSE PRACTITIONER            207
LICENSED VOCATIONAL NURSE     194
Public Health Nurse           177
NURSE MANAGER                  80
Clinical Nurse Specialist      68
PUBLIC HEALTH NURSE            58
Nurses Staffing Assistant      57
Nurse Midwife                  54
CLINICAL NURSE SPECIALIST      27
NURSES STAFFING ASSISTANT      21
NURSE MIDWIFE                  18
Nursery Specialist             14
NURSERY SPECIALIST              5
Chief Nursery Specialist        3
OPERATING ROOM NURSE            2
Operating Room Nurse            2
CHIEF NURSERY SPECIALIST        1
Name: JobTitle, dtype: Int64

In [18]:
sf_data.loc[sf_data['JobTitle'].str.lower().str.contains('police officer'), 'JobTitle'].value_counts()

# Here as earlier these should be grouped into one Jobtitle.

Police Officer 3                2421
Police Officer                  1476
Police Officer 2                1141
POLICE OFFICER III               779
POLICE OFFICER I                 467
POLICE OFFICER II                361
Institutional Police Officer      37
INSTITUTIONAL POLICE OFFICER      13
Name: JobTitle, dtype: Int64

I'm going to create a new column "JobTitle_New". 

In this column, I'm grouping appropriate Jobtitles in existing column "JobTitle".

Only the top few Jobtitles are grouped because it takes a lot of time to group the rest.

In [19]:
def JobTitle_New(job):
    if bool(re.search('nurse\s|\snurse\s|\snurse$|nurses\s', job.lower())):
        return 'Nurse'
    elif job.lower().__contains__('transit operator'):
        return 'Transit Operator'
    elif job.lower().__contains__('clerk'):
        return 'Clerk'
    elif job.lower().__contains__('police'):
        return 'Police'
    elif job.lower().__contains__('firefighter'):
        return 'Firefighter'
    elif job.lower().__contains__('engineer'):
        return 'Engineer'
    elif job.lower().__contains__('custodian'):
        return 'Custodian'
    elif bool(re.search('sheriff\s|\ssheriff$|.sheriff|^sheriff$', job.lower())):
        return 'Sheriff'
    elif job.lower().__contains__('eligibility worker'):
        return 'Eligibility Worker'
    elif job.lower().__contains__('recreation leader'):
        return 'Recreation Leader'
    elif job.lower().__contains__('public svc aide-public works'):
        return 'Public Svc Aide-Public Works'
    elif job.lower().__contains__('guard'):
        return 'Guard'
    elif job.lower().__contains__('administrative analyst'):
        return 'Administrative Analyst'
    elif job.lower().__contains__('patient care assistant'):
        return 'Patient Care Assistant'
    elif job.lower().__contains__('sergeant'):
        return 'Sergeant'
    elif job.lower().__contains__('attorney'):
        return 'Attorney'
    elif job.lower().__contains__('public service trainee'):
        return 'Public Service Trainee'
    elif job.lower().__contains__('general laborer'):
        return 'General Laborer'
    elif bool(re.search('^porter$|^porter\s', job.lower())):
        return 'Porter'
    elif job.lower().__contains__('gardener'):
        return 'Gardener'
    elif job.lower().__contains__('parking control officer'):
        return 'Parking Control Officer'
    elif job.lower().__contains__('library'):
        return 'Library Page & Assistant'
    elif job.lower().__contains__('food service'):
        return 'Food Service'
    elif job.lower().__contains__('public service'):
        return 'Public Service'
    elif job.lower().__contains__('librarian'):
        return 'Librarian'
    elif job.lower().__contains__('truck driver'):
        return 'Truck Driver'
    elif job.lower().__contains__('safety'):
        return 'Public Safety Officer'
    elif job.lower().__contains__('social worker'):
        return 'Social Worker'
    elif job.lower().__contains__('medical'):
        return 'Medical Staff'
    
sf_data['JobTitle_New'] = sf_data['JobTitle'].map(JobTitle_New)

In [20]:
# Rest of the rows in column "JobTitle_New" is filled with Jobtitle in "JobTitle" column. Jobtitle string is Capitalised to make it unifrom.
sf_data.loc[sf_data['JobTitle_New'].isna(), 'JobTitle_New'] = \
sf_data.loc[sf_data['JobTitle_New'].isna(), 'JobTitle_New'].fillna((sf_data['JobTitle']).str.capitalize())

In [21]:
# Top 50 Jobtitles and Counts in new column "JobTitle_New"
sf_data['JobTitle_New'].value_counts().head(50)

Nurse                                  13354
Transit Operator                        9424
Clerk                                   9353
Police                                  8201
Engineer                                4751
Firefighter                             4071
Custodian                               3214
Sheriff                                 3085
Eligibility Worker                      2723
Recreation Leader                       2663
Public Svc Aide-Public Works            2518
Guard                                   2193
Administrative Analyst                  2177
Library Page & Assistant                1950
Patient Care Assistant                  1945
Attorney                                1841
Social Worker                           1748
Public Service Trainee                  1656
General Laborer                         1593
Porter                                  1524
Sergeant                                1386
Gardener                                1274
Parking Co

In [22]:
# Converting data type of column "JobTitle_New".
sf_data['JobTitle_New'] = sf_data['JobTitle_New'].astype('string')

In [23]:
# Number of Null values in each columns.
sf_data.isna().sum()


Id                      0
EmployeeName            0
JobTitle                0
BasePay               609
OvertimePay             4
OtherPay                4
Benefits            36163
TotalPay                0
TotalPayBenefits        0
Year                    0
Agency                  0
JobTitle_New            0
dtype: int64

In [24]:
# Total Rows & Columns.
sf_data.shape

(148654, 12)

In [25]:
# The Rows in which "JobTitle" column does not have any Jobtitle.
sf_data.loc[sf_data['JobTitle'] == "Not provided"]

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Agency,JobTitle_New
148646,148647,Not provided,Not provided,NaN,NaN,NaN,NaN,0.0,0.0,2014,San Francisco,Not provided
148650,148651,Not provided,Not provided,NaN,NaN,NaN,NaN,0.0,0.0,2014,San Francisco,Not provided
148651,148652,Not provided,Not provided,NaN,NaN,NaN,NaN,0.0,0.0,2014,San Francisco,Not provided
148652,148653,Not provided,Not provided,NaN,NaN,NaN,NaN,0.0,0.0,2014,San Francisco,Not provided


In [26]:
# Removing the Rows in which "JobTitle" column does not have any Jobtitle.
sf_data = sf_data.loc[sf_data['JobTitle'] != "Not provided"]

In [27]:
# Number of Null values in each columns.
sf_data.isna().sum()

Id                      0
EmployeeName            0
JobTitle                0
BasePay               605
OvertimePay             0
OtherPay                0
Benefits            36159
TotalPay                0
TotalPayBenefits        0
Year                    0
Agency                  0
JobTitle_New            0
dtype: int64

In [28]:
sf_data = sf_data.loc[sf_data['JobTitle'] != "Not provided"]

In [29]:
sf_data[sf_data['BasePay'] < 0]

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Agency,JobTitle_New
72832,72833,Irwin Sidharta,Junior Clerk,-166.01,249.02,0.0,6.56,83.010002,89.570000,2012,San Francisco,Clerk
72865,72866,Robert Scott,Junior Clerk,-121.63,182.70,0.0,5.44,61.070000,66.510002,2012,San Francisco,Clerk
72872,72873,Chung Huey Kung,Junior Clerk,-109.22,163.83,0.0,4.32,54.610001,58.930000,2012,San Francisco,Clerk
72874,72875,Jordan Li,Junior Clerk,-106.60,159.90,0.0,4.66,53.299999,57.959999,2012,San Francisco,Clerk
72878,72879,Richard Jackson,Junior Clerk,-101.88,153.08,0.0,4.55,51.200001,55.750000,2012,San Francisco,Clerk
72884,72885,DiMarco McGhee-Stewart,Junior Clerk,-93.14,139.97,0.0,4.17,46.830002,51.000000,2012,San Francisco,Clerk
72888,72889,Leopoldo Marasigan,Junior Clerk,-87.38,131.06,0.0,3.89,43.680000,47.570000,2012,San Francisco,Clerk
72894,72895,Douglas Avalos,Junior Clerk,-75.67,113.76,0.0,3.39,38.090000,41.480000,2012,San Francisco,Clerk
72908,72909,Norma Rodriguez,Junior Clerk,-59.59,89.65,0.0,2.68,30.059999,32.740002,2012,San Francisco,Clerk
72920,72921,Charles Williams,Junior Clerk,-30.58,45.87,0.0,1.36,15.290000,16.650000,2012,San Francisco,Clerk


In [30]:
# These rows contain Total Pay Benefits as negative values which is not possible.
sf_data.loc[sf_data['TotalPayBenefits'] < 0]

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Agency,JobTitle_New
110528,110529,Timothy E Gibson,Police Officer 3,NaN,0.0,0.00,-2.73,0.000000,-2.730000,2013,San Francisco,Police
110529,110530,Mark E Laherty,Police Officer 3,NaN,0.0,0.00,-8.20,0.000000,-8.200000,2013,San Francisco,Police
110530,110531,David P Kucia,Police Officer 3,NaN,0.0,0.00,-33.89,0.000000,-33.889999,2013,San Francisco,Police
148653,148654,Joe Lopez,"Counselor, Log Cabin Ranch",0.0,0.0,-618.13,0.00,-618.130005,-618.130005,2014,San Francisco,"Counselor, log cabin ranch"


In [31]:
# Removing the rows containing Total Pay Benefits as negative values.
sf_data = sf_data.loc[sf_data['TotalPayBenefits'] >= 0]

In [32]:
# Number of Null values in each columns.
sf_data.isna().sum()

Id                      0
EmployeeName            0
JobTitle                0
BasePay               602
OvertimePay             0
OtherPay                0
Benefits            36159
TotalPay                0
TotalPayBenefits        0
Year                    0
Agency                  0
JobTitle_New            0
dtype: int64

In [33]:
# Filling the Null values in "Benefits" column.
# The "Benefits" column can be calculated by Subtracting "Total Pay" column from "Total Pay Benefits" column.

sf_data.loc[sf_data['Benefits'].isna(), 'Benefits'] = \
sf_data.loc[sf_data['Benefits'].isna(), 'Benefits'].fillna(sf_data['TotalPayBenefits'] - sf_data['TotalPay'])

In [34]:
# Number of Null values in each columns.
sf_data.isna().sum()

Id                    0
EmployeeName          0
JobTitle              0
BasePay             602
OvertimePay           0
OtherPay              0
Benefits              0
TotalPay              0
TotalPayBenefits      0
Year                  0
Agency                0
JobTitle_New          0
dtype: int64

In [35]:
# Filling the Null values in "BasePay" column.
# The "BasePay" column can be calculated by Subtracting "OvertimePay" column and "OtherPay" column from "Total Pay" column.
sf_data.loc[sf_data['BasePay'].isna(), 'BasePay'] = \
sf_data.loc[sf_data['BasePay'].isna(), 'BasePay'].fillna(round(sf_data['TotalPay'] - sf_data['OtherPay'] - sf_data['OvertimePay'],0).abs())

In [36]:
# Number of Null values in each columns.
sf_data.isna().sum()

Id                  0
EmployeeName        0
JobTitle            0
BasePay             0
OvertimePay         0
OtherPay            0
Benefits            0
TotalPay            0
TotalPayBenefits    0
Year                0
Agency              0
JobTitle_New        0
dtype: int64

In [37]:
sf_data.head()

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Agency,JobTitle_New
0,1,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18,0.00,400184.25,0.0,567595.43750,567595.43750,2011,San Francisco,General manager-metropolitan transit authority
1,2,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02,245131.88,137811.38,0.0,538909.25000,538909.25000,2011,San Francisco,Police
2,3,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739.13,106088.18,16452.60,0.0,335279.90625,335279.90625,2011,San Francisco,Police
3,4,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916.00,56120.71,198306.90,0.0,332343.62500,332343.62500,2011,San Francisco,Wire rope cable maintenance mechanic
4,5,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134401.60,9737.00,182234.59,0.0,326373.18750,326373.18750,2011,San Francisco,"Deputy chief of department,(fire department)"


In [38]:
summary_statistic(sf_data)

,feature,type,count_unique,count_nan,count,mean,std,min,25%,50%,75%,max
0,Id,int32,148646,0,148646,74324.269324,42911.539223,1.00,37162.250000,74323.500000,111487.750000,148650.0000
1,EmployeeName,string,110806,0,148646,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,JobTitle,string,2158,0,148646,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BasePay,float64,109489,0,148646,66057.284243,42885.127379,-166.01,32807.347500,64863.905000,94611.000000,319275.0100
4,OvertimePay,float64,65998,0,148646,5066.196212,11454.504527,-0.01,0.000000,0.000000,4658.447500,245131.8800
5,OtherPay,float64,83224,0,148646,3648.869642,8056.685988,-7058.59,0.000000,811.350000,4236.575000,400184.2500
6,Benefits,float64,98462,0,148646,18925.251626,17165.179616,-13.80,0.000000,23219.800000,33469.002500,96570.6600
7,TotalPay,float32,138317,0,148646,74772.132812,50514.109375,0.00,36172.766602,71427.648438,105845.541016,567595.4375
8,TotalPayBenefits,float32,141805,0,148646,93698.101562,62790.914062,0.00,44081.410156,92406.304688,132879.820312,567595.4375
9,Year,int64,4,0,148646,2012.522584,1.117533,2011.00,2012.000000,2013.000000,2014.000000,2014.0000


In [39]:
sf_data

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Agency,JobTitle_New
0,1,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18,0.00,400184.25,0.0,567595.43750,567595.43750,2011,San Francisco,General manager-metropolitan transit authority
1,2,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02,245131.88,137811.38,0.0,538909.25000,538909.25000,2011,San Francisco,Police
2,3,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739.13,106088.18,16452.60,0.0,335279.90625,335279.90625,2011,San Francisco,Police
3,4,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916.00,56120.71,198306.90,0.0,332343.62500,332343.62500,2011,San Francisco,Wire rope cable maintenance mechanic
4,5,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134401.60,9737.00,182234.59,0.0,326373.18750,326373.18750,2011,San Francisco,"Deputy chief of department,(fire department)"
...,...,...,...,...,...,...,...,...,...,...,...,...
148644,148645,Randy D Winn,"Stationary Eng, Sewage Plant",0.00,0.00,0.00,0.0,0.00000,0.00000,2014,San Francisco,"Stationary eng, sewage plant"
148645,148646,Carolyn A Wilson,Human Services Technician,0.00,0.00,0.00,0.0,0.00000,0.00000,2014,San Francisco,Human services technician
148647,148648,Joann Anderson,Communications Dispatcher 2,0.00,0.00,0.00,0.0,0.00000,0.00000,2014,San Francisco,Communications dispatcher 2
148648,148649,Leon Walker,Custodian,0.00,0.00,0.00,0.0,0.00000,0.00000,2014,San Francisco,Custodian
